# Multi-Layer Perceptron

In [2]:
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report

%matplotlib inline

In [3]:
def write_predictions(predictions, file_name='pred.csv'):
    with open(file_name, 'w') as file:
        file.write('ID,Label')
        for index, value in enumerate(predictions):
            file.write('\n{0},{1}'.format(index+1, value))

In [4]:
# Load Data
X_train = pd.read_csv('Train/trainVectors.csv', header=None).transpose()
y_train = pd.read_csv('Train/trainLbls.csv', header=None, names=['label'])['label']
X_validation = pd.read_csv('Validation/valVectors.csv', header=None).transpose()
y_validation = pd.read_csv('Validation/valLbls.csv', header=None, names=['label'])['label']
X_test = pd.read_csv('Test/testVectors.csv', header=None).transpose()

In [5]:
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline


Using TensorFlow backend.


In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)


In [7]:
num_classes = len(y_train.unique())
num_dimensions = X_train.shape[1]

def baseline_model():
    # define baseline model
    model = Sequential()
    model.add(Dense(10, input_dim=num_dimensions, activation='tanh'))
    model.add(Dropout(.5))
    #model.add(Dense(20, input_dim=num_dimensions, activation='tanh'))
    #model.add(Dense(50, input_dim=num_dimensions, activation='elu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

def plot_perf(keras_history):
    loss_values = keras_history.history['loss']
    val_loss_values = keras_history.history['val_loss']
    accuracy = keras_history.history['acc']
    epochs = range(1, len(loss_values) + 1)

    # Plot the scores
    fig, ax = plt.subplots(figsize=(4,4))
    ax.plot(epochs, loss_values, 'o-')
    ax.plot(epochs, val_loss_values, '-')
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Loss')

def plot_accuracy(keras_history):
    accuracy = keras_history.history['acc']
    epochs = range(1, len(accuracy) + 1)

    # Plot the scores
    fig, ax = plt.subplots(figsize=(4,4))
    ax.plot(epochs, accuracy, 'o-')
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Accuracy')
    

In [8]:
baseline_model().summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                40970     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 29)                319       
Total params: 41,289
Trainable params: 41,289
Non-trainable params: 0
_________________________________________________________________


In [9]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=10, batch_size=32, verbose=0)

In [10]:
checkpoint = ModelCheckpoint('best-model.h5', verbose=1, monitor='val_loss', save_best_only=True, mode='auto')
hist = estimator.fit(X_train, y_train, shuffle=True, validation_data=(X_validation, y_validation), callbacks=[checkpoint], verbose=1)

Train on 5830 samples, validate on 2298 samples
Epoch 1/10


InternalError: Blas GEMM launch failed : a.shape=(32, 4096), b.shape=(4096, 10), m=32, n=10, k=4096
	 [[Node: dense_3/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_dense_3_input_0_0/_51, dense_3/kernel/read)]]
	 [[Node: loss_1/mul/_89 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_388_loss_1/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'dense_3/MatMul', defined at:
  File "C:\Users\omar\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\omar\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\omar\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\User

In [258]:
estimator.model.load_weights('best-model.h5')

In [ ]:
plot_perf(hist)

In [ ]:
plot_accuracy(hist)

In [ ]:
y_validation_predictions = estimator.predict(X_validation)
print(classification_report(y_validation, y_validation_predictions))

## Using Cross Validation

In [6]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=20, batch_size=100, verbose=1)

In [7]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

In [8]:
results = cross_val_score(estimator, X_train, y_train, cv=kfold)

Epoch 1/20
5247/5247 [==============================] - 3s 559us/step - loss: 1.0560 - acc: 0.7263
Epoch 2/20
5247/5247 [==============================] - 2s 339us/step - loss: 0.2161 - acc: 0.9253
Epoch 3/20
5247/5247 [==============================] - 2s 320us/step - loss: 0.1352 - acc: 0.9560
Epoch 4/20
5247/5247 [==============================] - 2s 315us/step - loss: 0.0497 - acc: 0.9853
Epoch 5/20
5247/5247 [==============================] - 2s 308us/step - loss: 0.0503 - acc: 0.9857
Epoch 6/20
5247/5247 [==============================] - 2s 331us/step - loss: 0.0262 - acc: 0.9929
Epoch 7/20
5247/5247 [==============================] - 2s 320us/step - loss: 0.0188 - acc: 0.9945
Epoch 8/20
5247/5247 [==============================] - 2s 328us/step - loss: 0.0222 - acc: 0.9947
Epoch 9/20
5247/5247 [==============================] - 2s 327us/step - loss: 0.0189 - acc: 0.9950
Epoch 10/20
5247/5247 [==============================] - 2s 337us/step - loss: 0.0211 - acc: 0.9943
Epoch 11/

5247/5247 [==============================] - 2s 432us/step - loss: 0.9986 - acc: 0.7358
Epoch 2/20
5247/5247 [==============================] - 2s 328us/step - loss: 0.2290 - acc: 0.9240
Epoch 3/20
5247/5247 [==============================] - 2s 321us/step - loss: 0.1146 - acc: 0.9634
Epoch 4/20
5247/5247 [==============================] - 2s 316us/step - loss: 0.0614 - acc: 0.9846
Epoch 5/20
5247/5247 [==============================] - 2s 328us/step - loss: 0.0620 - acc: 0.9775
Epoch 6/20
5247/5247 [==============================] - 2s 314us/step - loss: 0.0424 - acc: 0.9886
Epoch 7/20
5247/5247 [==============================] - 2s 325us/step - loss: 0.0414 - acc: 0.9891
Epoch 8/20
5247/5247 [==============================] - 2s 322us/step - loss: 0.0178 - acc: 0.9954
Epoch 9/20
5247/5247 [==============================] - 2s 319us/step - loss: 0.0128 - acc: 0.9968
Epoch 10/20
5247/5247 [==============================] - 2s 316us/step - loss: 0.0084 - acc: 0.9987
Epoch 11/20
5247/524

ResourceExhaustedError: OOM when allocating tensor with shape[4096,2062] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_9/Adam/Square = Square[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_9/Adam/gradients/dense_19/MatMul_grad/MatMul_1)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'training_9/Adam/Square', defined at:
  File "C:\Users\omar\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\omar\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\omar\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\omar\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C

In [ ]:
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
y_validation_predictions = estimator.predict(X_validation)
print(classification_report(y_validation, y_validation_predictions))

In [9]:
baseline_model().summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 2062)              8448014   
_________________________________________________________________
dense_22 (Dense)             (None, 29)                59827     
Total params: 8,507,841
Trainable params: 8,507,841
Non-trainable params: 0
_________________________________________________________________


In [8]:
hist = estimator.fit(X_train, y_train, shuffle=True, validation_split=0.2, verbose=1)

Train on 4664 samples, validate on 1166 samples
Epoch 1/20
 - 120s - loss: 3.9904 - acc: 0.5971 - val_loss: 14.2498 - val_acc: 0.0489
Epoch 2/20
 - 119s - loss: 0.6574 - acc: 0.8422 - val_loss: 14.7031 - val_acc: 0.0480
Epoch 3/20
 - 119s - loss: 0.4850 - acc: 0.8885 - val_loss: 14.8774 - val_acc: 0.0515
Epoch 4/20
 - 119s - loss: 0.3432 - acc: 0.9237 - val_loss: 15.0133 - val_acc: 0.0463
Epoch 5/20
 - 122s - loss: 0.3266 - acc: 0.9363 - val_loss: 14.9705 - val_acc: 0.0429
Epoch 6/20
 - 120s - loss: 0.2952 - acc: 0.9470 - val_loss: 15.0774 - val_acc: 0.0549
Epoch 7/20
 - 119s - loss: 0.2562 - acc: 0.9543 - val_loss: 15.2387 - val_acc: 0.0437
Epoch 8/20
 - 120s - loss: 0.2755 - acc: 0.9552 - val_loss: 15.1056 - val_acc: 0.0575
Epoch 9/20
 - 119s - loss: 0.3266 - acc: 0.9539 - val_loss: 15.1964 - val_acc: 0.0497
Epoch 10/20
 - 121s - loss: 0.2394 - acc: 0.9648 - val_loss: 15.3161 - val_acc: 0.0420
Epoch 11/20
 - 120s - loss: 0.2999 - acc: 0.9584 - val_loss: 15.2860 - val_acc: 0.0420
Epoc

In [9]:
estimator.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2062)              8448014   
_________________________________________________________________
dense_2 (Dense)              (None, 29)                59827     
Total params: 8,507,841
Trainable params: 8,507,841
Non-trainable params: 0
_________________________________________________________________


In [12]:
#estimator.model.evaluate(X_validation, keras.utils.to_categorical(y_validation), verbose=1)

In [15]:
y_validation_predictions = estimator.predict(X_validation)
print(classification_report(y_validation, y_validation_predictions))

In [18]:
print(classification_report(y_validation, y_validation_predictions))

             precision    recall  f1-score   support

          1       0.97      1.00      0.98       116
          2       0.62      0.58      0.60        96
          3       0.73      0.70      0.72        94
          4       0.84      0.85      0.84        92
          5       0.74      0.83      0.78        88
          6       0.38      0.59      0.46        92
          7       0.57      0.40      0.47        92
          8       0.51      0.60      0.55        88
          9       0.75      0.77      0.76        88
         10       0.48      0.72      0.58        82
         11       0.86      0.92      0.89        86
         12       0.67      0.72      0.70        80
         13       0.66      0.96      0.78        80
         14       0.38      0.80      0.52        82
         15       0.40      0.80      0.54        82
         16       0.49      0.87      0.63        82
         17       0.84      0.90      0.87        80
         18       0.92      0.74      0.82   

/home/omar/anaconda3/envs/cvml/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
